In [4]:
import pandas as pd
from bs4 import BeautifulSoup
import pymorphy2
import re
import numpy as np

In [ ]:
train_data = pd.read_csv('data/train.csv', sep='\t', index_col='id')

## Описание полей
 * __id__ &mdash; внутренний идетификатор
 * __name__ &mdash; название вакансии
 * __description__ &mdash; текст вакансии
 * __target__ &mdash; класс заинтересованности

In [ ]:
train_data.head()

In [ ]:
posConv={'ADJF':'_ADJ','NOUN':'_NOUN','VERB':'_VERB', 'PRTF': '_PRT', 'GRND': '_GRND'}

def getVacancyVector(text, needPos=None):
    text = BeautifulSoup(text, 'lxml').get_text()
    
    words=[a[0] for a in re.findall("([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)", text)]
    reswords=[]

    for w in words:
        wordform=morph.parse(w)[0]
        if wordform.tag.POS in ['ADJF', 'NOUN', 'VERB', 'PRTF', 'GRND']:
            if needPos!=None:
                reswords.append(wordform.normal_form+posConv[wordform.tag.POS])
            else:
                reswords.append(wordform.normal_form)
            
    return reswords  


In [ ]:
morph=pymorphy2.MorphAnalyzer() 

Для начала обработаем все записи и сохраним обработанную таблицу, что бы в будущем обращатся к размеченным данным.

In [ ]:
%%time

train_data.description = train_data.apply(lambda row: getVacancyVector(row['description'], True), axis=1)
train_data.name = train_data.apply(lambda row: getVacancyVector(row['name'], True), axis=1)

## Начинаем непосредственно обучение

In [32]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

### CountVectorizer + TF_IDF + LogisticClassifier без лемматизации

In [33]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [5]:
train = pd.read_csv('data/train.csv', sep='\t', index_col='id')
test = pd.read_csv('data/test.csv', sep='\t', index_col='id')

In [6]:
test_array = np.array(test['name']+' '+test['description'])


In [44]:
train_array = np.array(train['name']+' '+train['description'])
train_array.shape

(200000,)

In [125]:
texts = []
for t in np.hstack((train_array, test_array)):    
    texts.append(
        BeautifulSoup(t, 'lxml').text
    )

In [126]:
y = train.iloc[:, -1].values

In [128]:
cv = CountVectorizer(max_df=0.8, min_df=3)

In [131]:
temp_matrix = cv.fit_transform(texts)

In [132]:
tfidf_transformer = TfidfTransformer(norm=None, smooth_idf=False)
matrix_tfidf = tfidf_transformer.fit_transform(temp_matrix)

In [133]:
classifier = LogisticRegression(random_state=123, n_jobs=2)

In [ ]:
%%time 

classifier.fit(matrix_tfidf[:train_array.shape[0]], y)

C:\Users\unifam.OFFICE\AppData\Local\Continuum\anaconda2\envs\p36\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(self.n_jobs))


In [71]:
%%time 
predictions = classifier.predict(matrix_tfidf[train_array.shape[0]:])

Wall time: 404 ms


In [104]:
result1 = pd.DataFrame(predictions, columns=['target'], index=np.arange(200000, 370179))

In [106]:
result1.to_csv('data/result1.csv', index_label='id')

In [ ]:
from sklearn.cross_validation import cross_val_score
cross_val_score(
    classifier,
    matrix_tfidf[:train_array.shape[0]],
    y,
    scoring='roc_auc',
    cv=5
)

In [ ]:
rfc = RandomForestClassifier()

In [ ]:
model = Pipeline (
    [
        ('cv', CountVectorizer()),
        ('tfidf', TfidfTransformer(norm=None, smooth_idf=False)),
    ]
)

In [ ]:
from gensim.models.word2vec import Word2Vec # Собственно модель.
from gensim.models.word2vec import LineSentence # Выравнивание текста по предложениям.
from gensim.models import KeyedVectors # Семантические вектора.